In [2]:
import os
import sys
sys.path.append('../Module')
folder_path = '../Dataset'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score, classification_report, accuracy_score
from sklearn import set_config

from catboost import CatBoostClassifier

from data_to_csv import df_to_csv
from utils import generate_description

df_0 = pd.read_csv(folder_path + '/SBA_Cleaned_0.csv')
df_1 = pd.read_csv(folder_path + '/SBA_Cleaned_1.csv')
df_2 = pd.read_csv(folder_path + '/SBA_Cleaned_2.csv')
df_pure = pd.read_csv(folder_path + '/SBA_Cleaned_Pure.csv')

In [3]:
df_pure.dropna(inplace=True)
generate_description(df_pure)

,feature,data_type,null,nulPct,unique,uniqueSample
0,City,object,0,0.0,28860,"[FLORISSANT, ANDERSON, SOUTH SAN FRANCISCO, FL..."
1,State,object,0,0.0,51,"[OH, GA, AZ, RI, PA]"
2,Bank,object,0,0.0,5221,"[BANK OF AMERICA NATL ASSOC, HANMI BANK, READY..."
3,BankState,object,0,0.0,55,"[IL, CA, NH, OR]"
4,NAICS,object,0,0.0,20,"[23, 44-45, 48-49, 72]"
5,ApprovalDate,object,0,0.0,7367,"[14-03-07, 13-04-05, 19-06-03, 06-05-04, 23-11..."
6,ApprovalFY,int64,0,0.0,29,"[2007, 2001, 2002, 1998, 1994]"
7,Term,int64,0,0.0,378,"[72, 84, 57, 120, 228]"
8,NoEmp,int64,0,0.0,502,"[7, 5, 12, 6]"
9,NewExist,bool,0,0.0,2,[True]


In [4]:
y = df_pure['Approve']
X = df_pure.drop('Approve', axis=1)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.05, random_state=42, stratify=y)

In [8]:
categorical_columns = X.select_dtypes(include='object').columns
model = CatBoostClassifier(ategorical_columnsr = categorical_columns)
model.fit(X_train,y_train)

TypeError: CatBoostClassifier.__init__() got an unexpected keyword argument 'ategorical_columns_selector'

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy : {accuracy}')

f1_macro = f1_score(y_test, y_pred, average="macro")
print(f"Score F1 : {f1_macro}")

print(classification_report(y_test, y_pred))

In [ ]:
numerical_columns_selector = X.select_dtypes(exclude='object').columns
categorical_columns_selector = X.select_dtypes(include='object').columns

numerical_columns = X[numerical_columns_selector]
categorical_columns = X[categorical_columns_selector]

categorical_preprocessor = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

numerical_preprocessor = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ("nums", numerical_preprocessor, numerical_columns_selector),
        ("cat", categorical_preprocessor, categorical_columns_selector)
    ],
    remainder='passthrough'
)

In [ ]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', CatBoostClassifier(random_state=42, verbose=0))
])

param_grid = {
    'model__n_estimators': [100, 200],  
    'model__max_depth': [5, 10],  
    'model__learning_rate': [0.1, 0.01],  
    'model__subsample': [0.8, 0.9],  
    'model__colsample_bylevel': [0.8, 0.9]  
}

/home/utilisateur/projet/Brief_Prêt/Model_pret/.venv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:988: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/utilisateur/projet/Brief_Prêt/Model_pret/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py", line 113, in _check_targets
    unique_values = _union1d(y_true, y_pred, xp)
  File "/home/utilisateur/projet/Brief_Prêt/Model_pret/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py", line 118, in _union1d
    return xp.asarray(numpy.union1d(a, b))
  File "/home/utilisateur/projet/Brief_Prêt/Model_pret/.venv/lib/python3.10/site-packages/numpy/lib/arraysetops.py", line 932, in union1d
    return unique(np.concatenate((ar1, ar2), axis=None))
  File "/home/utilisateur/projet/Brief_Prêt/Model_pret/.venv/lib/python3.10/site-packages/numpy/lib/arraysetops.py", line 274, in unique

In [ ]:
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='f1_macro')
grid_search.fit(X_train, y_train)

print(f"Best hyperparameters : {grid_search.best_params_}")
print(f"Best accuracy : {grid_search.best_score_}")

In [ ]:
best_model = grid_search.best_estimator_
predictions = best_model.predict(X_test)

In [ ]:
y_pred = best_model.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy : {accuracy}')

In [ ]:
f1_macro = f1_score(y_test, y_pred, average="macro")
print(f"Score F1 : {f1_macro}")

In [ ]:
print(classification_report(y_test, y_pred))